# Topic 3: Recommender System

## Business Objective/Problem
- Tiki là một hệ sinh thái thương mại “all in one”, trong
đó có tiki.vn, là một website thương mại điện tử đứng
top 2 của Việt Nam, top 6 khu vực Đông Nam Á.
- Trên trang này đã triển khai nhiều tiện ích hỗ trợ nâng
cao trải nghiệm người dùng và họ muốn xây dựng
nhiều tiện ích hơn nữa.
- Giả sử công ty này chưa triển khai Recommender
System và bạn được yêu cầu triển khai hệ thống này,
bạn sẽ làm gì?

## Các kiến thức/ kỹ năng cần để giải quyết
vấn đề này:
- Hiểu vấn đề
- Import các thư viện cần thiết và hiểu cách sử dụng
- Đọc dữ liệu (dữ liệu project này được cung cấp)
- Thực hiện EDA cơ bản (sử dụng Pandas Profiling
Report)
- Tiền xử lý dữ liệu: làm sạch, tạo tính năng mới, lựa
chọn tính năng cần thiết…

## Bước 1: Business Understanding
- Dựa vào yêu cầu nói trên => xác định vấn đề:
  - Chưa có hệ thống Recommendation System
  - => Mục tiêu/ vấn đề: Xây dựng Recommendation System
  cho một hoặc một số nhóm hàng hóa trên tiki.vn giúp đề
  xuất và gợi ý cho người dùng/ khách hàng. => Xây dựng
  các mô hình đề xuất:
    - Content-based filtering
    - Collaborative filtering

Bước 2: Data Understanding/ Acquire
- Từ mục tiêu/ vấn đề đã xác định: xem xét các dữ
liệu cần thiết:
  - Dữ liệu được cung cấp sẵn gồm có các tập tin:
  ProductRaw.csv, ReviewRaw.csv chứa thông tin sản phẩm,
  review và rating cho các sản phẩm thuộc các nhóm hàng
  hóa như Mobile_Tablet, TV_Audio, Laptop, Camera,
  Accessory

# Bước 3: Data preparation/ Prepare

### 1. PREPROCESSING

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd '/content/gdrive/My Drive/LDS0_K271_PhamThiHoa/Project_3'

/content/gdrive/My Drive/LDS0_K271_PhamThiHoa/Project_3


In [ ]:
import pandas as pd
import numpy as np

# 1. Products

In [ ]:
products = pd.read_csv('Data/ProductRaw.csv')

In [ ]:
print(f'Row: {products.shape[0]}\nColumns: {products.shape[1]}')

Row: 4404
Columns: 10


In [ ]:
products.head(3)

,item_id,name,description,rating,price,list_price,brand,group,url,image
0,48102821,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...,THÔNG TIN CHI TIẾT\nDung lượng pin 300\nThời g...,4.0,77000,300000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://tai-nghe-bluetooth-inpods-12-cam-bien-...,https://salt.tikicdn.com/cache/280x280/ts/prod...
1,52333193,Tai nghe bluetooth không dây F9 True wireless ...,THÔNG TIN CHI TIẾT\nDung lượng pin 2000mah\nTh...,4.5,132000,750000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://tai-nghe-bluetooth-khong-day-f9-true-w...,https://salt.tikicdn.com/cache/280x280/ts/prod...
2,299461,Chuột Không Dây Logitech M331 Silent - Hàng Ch...,THÔNG TIN CHI TIẾT\nThương hiệu Logitech\nĐộ p...,4.8,299000,399000,Logitech,Thiết Bị Số - Phụ Kiện Số/Phụ kiện máy tính và...,https://chuot-khong-day-logitech-m331-silent-p...,https://salt.tikicdn.com/cache/280x280/media/c...


In [ ]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4404 entries, 0 to 4403
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   item_id      4404 non-null   int64  
 1   name         4404 non-null   object 
 2   description  4401 non-null   object 
 3   rating       4404 non-null   float64
 4   price        4404 non-null   int64  
 5   list_price   4404 non-null   int64  
 6   brand        4404 non-null   object 
 7   group        4404 non-null   object 
 8   url          4404 non-null   object 
 9   image        4404 non-null   object 
dtypes: float64(1), int64(3), object(6)
memory usage: 344.2+ KB


In [ ]:
# Check duplicate
print('Duplicate: ',products[products.duplicated()].shape[0])
products[products.duplicated()].head(3)

Duplicate:  31


,item_id,name,description,rating,price,list_price,brand,group,url,image
2002,356188,Chuột Có Dây Logitech B100 - Hàng Chính Hãng,THÔNG TIN CHI TIẾT\nThương hiệu Logitech\nKích...,4.7,58000,90000,Logitech,Thiết Bị Số - Phụ Kiện Số/Phụ kiện máy tính và...,https://chuot-co-day-logitech-b100-hang-chinh-...,https://salt.tikicdn.com/cache/280x280/ts/prod...
2006,646020,Bàn Phím Có Dây Dell KB216 - Đen - Hàng Chính ...,THÔNG TIN CHI TIẾT\nThương hiệu Dell\nKích thư...,4.6,169000,287500,Dell,Thiết Bị Số - Phụ Kiện Số/Phụ kiện máy tính và...,https://ban-phim-co-day-dell-kb216-den-hang-ch...,https://salt.tikicdn.com/cache/280x280/media/c...
2008,359484,Bàn Phím Có Dây Logitech K120 - Hàng Chính Hãng,THÔNG TIN CHI TIẾT\nThương hiệu Logitech\nKết ...,4.7,135000,209000,Logitech,Thiết Bị Số - Phụ Kiện Số/Phụ kiện máy tính và...,https://ban-phim-co-day-logitech-k120-hang-chi...,https://salt.tikicdn.com/cache/280x280/ts/prod...


In [ ]:
# Check for example item_id = 356188
products[products.item_id == 356188]

,item_id,name,description,rating,price,list_price,brand,group,url,image
14,356188,Chuột Có Dây Logitech B100 - Hàng Chính Hãng,THÔNG TIN CHI TIẾT\nThương hiệu Logitech\nKích...,4.7,58000,90000,Logitech,Thiết Bị Số - Phụ Kiện Số/Phụ kiện máy tính và...,https://chuot-co-day-logitech-b100-hang-chinh-...,https://salt.tikicdn.com/cache/280x280/ts/prod...
2002,356188,Chuột Có Dây Logitech B100 - Hàng Chính Hãng,THÔNG TIN CHI TIẾT\nThương hiệu Logitech\nKích...,4.7,58000,90000,Logitech,Thiết Bị Số - Phụ Kiện Số/Phụ kiện máy tính và...,https://chuot-co-day-logitech-b100-hang-chinh-...,https://salt.tikicdn.com/cache/280x280/ts/prod...


In [ ]:
# Delete duplicate:
print(f'Before drop duplicates: {products.shape[0]}')
products = products.drop_duplicates()
print(f'After drop duplicates: {products.shape[0]}')

Before drop duplicates: 4404
After drop duplicates: 4373


In [ ]:
# Check Null data
products.isnull().any()

item_id        False
name           False
description     True
rating         False
price          False
list_price     False
brand          False
group          False
url            False
image          False
dtype: bool

In [ ]:
products[products.description.isnull()]

,item_id,name,description,rating,price,list_price,brand,group,url,image
1424,75831435,Camera Ip EZVIZ C8C hồng ngoại không dây Full ...,NaN,0.0,1465000,3200000,EZVIZ,Máy Ảnh - Máy Quay Phim/Camera Giám Sát/Camera IP,https://camera-ip-ezviz-c8c-hong-ngoai-khong-d...,https://salt.tikicdn.com/cache/280x280/ts/prod...
2535,11607917,Màn Hình LED LG 27MK600M-B 27 inch Full HD (19...,NaN,0.0,4200000,5500000,LG,Laptop - Máy Vi Tính - Linh kiện/Thiết Bị Văn ...,https://man-hinh-led-lg-27mk600m-b-27-inch-ful...,https://salt.tikicdn.com/cache/280x280/ts/prod...
3294,16608046,Điện Thoại Di Động Forme A5 - Hãng phân phối c...,NaN,3.7,335000,389000,Forme,Điện Thoại - Máy Tính Bảng/Điện thoại phổ thông,https://dien-thoai-di-dong-forme-a5-hang-phan-...,https://salt.tikicdn.com/cache/280x280/ts/prod...


- Have 3 Null in description. However, We will keep that rows to insight about data in next step


### Save file

In [ ]:
products.to_csv('Data/Products.csv', index = False)

# 2. Reviews

In [ ]:
reviews = pd.read_csv('Data/ReviewRaw.csv')

In [ ]:
print(f'Row: {reviews.shape[0]}\nColumns: {reviews.shape[1]}')

Row: 364099
Columns: 8


In [ ]:
reviews.head()

,customer_id,product_id,name,full_name,created_time,rating,title,content
0,709310,10001012,Lân Nguyễn Hoàng,Lân Nguyễn Hoàng,NaN,3,Ko dùng đc thẻ nhớ,Lúcđầu quên thông tin nên dùng 512gb thì ko đc...
1,10701688,10001012,Nguyễn Khánh Hòa,Nguyễn Khánh Hòa,NaN,5,Cực kì hài lòng,"Tiki giao hàng nhanh. Sản phẩm đúng như mô tả,..."
2,11763074,10001012,Toàn Phạm Khánh,Toàn Phạm Khánh,2019-04-17 15:42:45,5,Cực kì hài lòng,"chất lượng camera rõ nét, chống mưa nắng tuyệt..."
3,9909549,10001012,Nguyen Quang Minh,NaN,NaN,5,Rất hài lòng,"Hàng được đóng gói cẩn thận, giao hàng nhanh ,..."
4,1827148,10001012,Phạm Bá Đức,Phạm Bá Đức,NaN,5,Cực kì hài lòng,"dễ cài đặt, chất lượng tốt, chế độ xem hồng ng..."


In [ ]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364099 entries, 0 to 364098
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   customer_id   364099 non-null  int64 
 1   product_id    364099 non-null  int64 
 2   name          363669 non-null  object
 3   full_name     329471 non-null  object
 4   created_time  117340 non-null  object
 5   rating        364099 non-null  int64 
 6   title         364070 non-null  object
 7   content       165794 non-null  object
dtypes: int64(3), object(5)
memory usage: 22.2+ MB


In [ ]:
# Check duplicate
print('Duplicate: ',reviews[reviews.duplicated()].shape[0])
reviews[reviews.duplicated()].head()

Duplicate:  2349


,customer_id,product_id,name,full_name,created_time,rating,title,content
1818,6903478,10062880,Anh Phan,Anh Phan,2018-04-10 13:38:49,5,Cực kì hài lòng,NaN
2016,7200218,1011253,Trương Hoàng Tấn,Trương Hoàng Tấn,2018-05-28 09:29:11,5,Cực kì hài lòng,NaN
2445,5389981,1011522,Bùi Xuân Khánh,Bùi Xuân Khánh,2017-06-21 10:25:40,4,Hài lòng,NaN
2773,657328,10282851,Nguyễn Tân,Nguyễn Tân,2015-06-08 23:59:45,3,Bình thường,NaN
2816,17524382,10282851,Trọng Nhân,Trọng Nhân,NaN,5,Cực kì hài lòng,NaN


In [ ]:
# Check for example customer_id = 6903478, product_id = 10062880
reviews[(reviews.customer_id == 6903478) & (reviews.product_id == 10062880)]

,customer_id,product_id,name,full_name,created_time,rating,title,content
1809,6903478,10062880,Anh Phan,Anh Phan,2018-04-10 13:38:49,5,Cực kì hài lòng,NaN
1818,6903478,10062880,Anh Phan,Anh Phan,2018-04-10 13:38:49,5,Cực kì hài lòng,NaN


In [ ]:
# Delete duplicate:
print(f'Before drop duplicates: {reviews.shape[0]}')
reviews = reviews.drop_duplicates()
print(f'After drop duplicates: {reviews.shape[0]}')

Before drop duplicates: 364099
After drop duplicates: 361750


In [ ]:
# Check Null data
reviews.isnull().any()

customer_id     False
product_id      False
name             True
full_name        True
created_time     True
rating          False
title            True
content          True
dtype: bool

In [ ]:
null = []
for i in reviews.columns:
  if (reviews[reviews[i].isnull()].shape[0])> 0:
    null.append({
        'Column': i,
        'Missing Vallue': reviews[reviews[i].isnull()].shape[0],
        'Percent': reviews[reviews[i].isnull()].shape[0]/reviews.shape[0]*100
    })
df = pd.DataFrame(null)
df

,Column,Missing Vallue,Percent
0,name,428,0.118314
1,full_name,34603,9.565446
2,created_time,245525,67.871458
3,title,29,0.008017
4,content,196064,54.198756


- created_time and content have a large number of empty data.
- Other columns have less empty data percentage.
- However, since the dataset will use the recommendation ALS method, we will keep only 3 columns customer_id, product_id, rating
- Now we will remove column created_time and column full_name (because there is column 'name' which is the same with full_name) other columns we can keep to insight into the data in next step

In [ ]:
reviews = reviews.drop(['created_time','full_name'], axis = 1)

In [ ]:
reviews

,customer_id,product_id,name,rating,title,content
0,709310,10001012,Lân Nguyễn Hoàng,3,Ko dùng đc thẻ nhớ,Lúcđầu quên thông tin nên dùng 512gb thì ko đc...
1,10701688,10001012,Nguyễn Khánh Hòa,5,Cực kì hài lòng,"Tiki giao hàng nhanh. Sản phẩm đúng như mô tả,..."
2,11763074,10001012,Toàn Phạm Khánh,5,Cực kì hài lòng,"chất lượng camera rõ nét, chống mưa nắng tuyệt..."
3,9909549,10001012,Nguyen Quang Minh,5,Rất hài lòng,"Hàng được đóng gói cẩn thận, giao hàng nhanh ,..."
4,1827148,10001012,Phạm Bá Đức,5,Cực kì hài lòng,"dễ cài đặt, chất lượng tốt, chế độ xem hồng ng..."
...,...,...,...,...,...,...
364094,16820553,9996258,Quang Hng,5,Cực kì hài lòng,NaN
364095,5888640,9996258,Phan Thanh Tú,5,Cực kì hài lòng,NaN
364096,12246414,9996258,Nguyên Hồng,5,Cực kì hài lòng,NaN
364097,14309619,9996258,Justin H Chiem,5,Cực kì hài lòng,NaN


In [ ]:
# Number review with product_id that do not exist (not match with item_id in products)
print('review with product_id that do not exist: ',reviews[~reviews['product_id'].isin(products['item_id'])].shape[0]) 
reviews[~reviews['product_id'].isin(products['item_id'])]

review with product_id that do not exist:  660


,customer_id,product_id,name,rating,title,content
156669,5687133,20845056,Lê Phương Tiến,4,Hài lòng,Hài lòng về máy sau 1 tuần dùng. Điểm trừ giao...
156670,8344149,20845056,Luu Tri Dung,5,Tuyệt vời,"Giao hàng đúng thời gian dự kiến, có 2 ngày. M..."
156671,10671105,20845056,Nguyễn Thu Hằng,5,Cực kì hài lòng,"Sản phẩm được đóng gói cẩn thận, pin dùng được..."
156683,373425,20845056,Quân Nguyễn,2,Nên mua ở ngoài,Mình mua kindle Paperwhite 2019 được tầm cỡ 2 ...
156691,1279789,20845056,Quân Trần,1,"Ship chậm, máy ko active được",Tôi nhận đc sản phẩm ko active được. Khi liên ...
...,...,...,...,...,...,...
192248,10449689,12706787,Hai An,4,Hài lòng,có dùng cho dòng điện thoại khác ngoài iphone ...
192249,6079274,12706787,Nguyễn trung hiếu,4,"Hàng nghe mượt, tốt",Hàng chuẩn! Nhưng mình đang phân vân ko biết 1...
192250,7474878,12706787,Mon Pham,5,Quá phê !!!!,Mong lần sau đừng giao hàng chậm nữa tháng như...
192251,13144258,12706787,Đỗ Đức Việt,5,Thắc mắc,Hàng nhập khẩu với hàng VNA khác nhau gì vậy T...


In [ ]:
# Remove reviews with product_id that do not exist
print('Before: ', reviews.shape[0])
reviews = reviews[reviews['product_id'].isin(products['item_id'])]
print('After: ', reviews.shape[0])

Before:  361750
After:  361090


In [ ]:
reviews.reset_index(drop = True, inplace = True)

#### Save data

In [ ]:
reviews.to_csv('Data/Reviews.csv',index= False)